In [41]:
import os
from pathlib import Path
os.chdir(Path("E:\FSDS_NOV\ML-WaferFault-Detection"))

In [54]:
from collections import namedtuple
from waferFaultDetection.constants import *
from waferFaultDetection.utils import read_yaml,create_directories
from waferFaultDetection import logger

In [43]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    '''
    Define the resources required for the data ingestion
    such as URL, dataset file name, directory details
    '''
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    
# DataIngestionConfig = namedtuple("DataIngestionConfig",[
#     "root_dir",
#     "source_URL",
#     "local_data_file",
#     "unzip_dir"
# ])

In [44]:
from waferFaultDetection import logger
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
        ):

        logger.info("reading yaml files for configs and parameters")
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        logger.info("creating directory for artifacts")
        create_directories([self.config.artifacts_root])
        logger.info("artifacts directory created")

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        logger.info("creating data ingestion root directory")
        create_directories([config.root_dir])

        logger.info('creating data ingestion configuration')
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config


In [48]:
import os
import urllib.request as request
from zipfile import ZipFile
import json
import kaggle
from waferFaultDetection import logger

class DataIngestion:
    def __init__(
        self,
        config:DataIngestionConfig
    ):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            try:
                logger.info("downloading a dataset ...")
                filename, headers = request.urlretrieve(
                    url = self.config.source_URL,
                    filename = self.config.local_data_file
                )
                logger.info('dataset download completed')
            except Exception as e:
                logger.info('dataset download failed')
                raise e

    def download_kaggle_dataset(self):
        if not os.path.exists(self.config.local_data_file):
            try:
                logger.info("downloading a dataset ...")
                # importing the module
                import json
                import os
                import kaggle

                # Opening JSON file
                with open(r'C:\Users\Najam\.kaggle\kaggle.json') as json_file:
                    data = json.load(json_file)

                os.environ['KAGGLE_USERNAME']=data['username']
                os.environ['KAGGLE_KEY']=data['key']


                from kaggle.api.kaggle_api_extended import KaggleApi
                api = KaggleApi()
                api.authenticate()
                api.dataset_download_files(self.config.source_URL, path=self.config.unzip_dir)
                logger.info("dataset download completed successfully")
            except Exception as e:
                logger.info('dataset download failed')
                raise e
    
    def _get_updated_list_of_files(self,list_of_files):
        logger.info('considers only .csv files from dataset zip file')
        return [f for f in list_of_files if f.endswith('.csv')]

    def _preprocess(self,zf:ZipFile,f:str,working_dir:str):
        target_filepath = os.path.join(working_dir,f)
        if not os.path.exists(target_filepath):
            zf.extract(f,working_dir)

        if os.path.getsize(target_filepath) == 0:
            os.remove(target_filepath)
    
    def unzip_and_clean(self):
        try:
            logger.info('unzipping dataset file started')
            with ZipFile(file=self.config.local_data_file,mode="r") as zf:
                list_of_files = zf.namelist()
                updated_list_of_files = self._get_updated_list_of_files(list_of_files)
                logger.info('pre-processing the dataset zip file and extraction starts')
                for f in updated_list_of_files:
                    self._preprocess(zf,f,self.config.unzip_dir)
                logger.info('zip file extraction completed')
        except Exception as e:
            raise e

In [52]:
from pathlib import Path

with ZipFile(file=Path('./artifacts/data_ingestion/waferdataset.zip'),mode="r") as zf:
    list_of_files = zf.namelist()
    # updated_list_of_files = self._get_updated_list_of_files(list_of_files)
    # for f in updated_list_of_files:
    #     self._preprocess(zf,f,self.config.unzip_dir)

In [53]:
list_of_files

['Prediction_Batch_files/Wafer_13012020_141000.csv',
 'Prediction_Batch_files/Wafer_14012020_113045.csv',
 'Prediction_Batch_files/Wafer_15010_130532.csv',
 'Prediction_Batch_files/wafer_07012020_041011.csv',
 'Prediction_Batch_files/wafer_13012020_090817.csv',
 'Prediction_Batch_files/wafer_16012020_051629.csv',
 'Prediction_Batch_files/wafer_20012020_090819.csv',
 'Prediction_Batch_files/wafer_20022020_090716.csv',
 'Prediction_Batch_files/wafer_21012020_080913.csv',
 'Prediction_Batch_files/wafer_22022020_041119.csv',
 'Prediction_Batch_files/wafer_23012020_011008.csv',
 'Prediction_Batch_files/wafer_23012020_041211.csv',
 'Prediction_Batch_files/wafer_27012020_080911.csv',
 'Prediction_Batch_files/wafer_28012020_051011.csv',
 'Prediction_Batch_files/wafer_28012020_090817.csv',
 'Prediction_Batch_files/wafer_28042020_031911.csv',
 'Prediction_Batch_files/wafer_29012020_050617.csv',
 'Prediction_Batch_files/wafer_29012020_060756.csv',
 'Prediction_Batch_files/wafer_31012020_090811.cs

In [49]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_kaggle_dataset()
    # data_ingestion.unzip_and_clean()
except Exception as e:
    raise e


[2022-11-17 22:06:15,173: INFO: common]: yaml file: configs\config.yaml loaded successfully
[2022-11-17 22:06:15,175: INFO: common]: yaml file: params.yaml loaded successfully
[2022-11-17 22:06:15,176: INFO: common]: created directory at: artifacts
[2022-11-17 22:06:15,178: INFO: common]: created directory at: artifacts/data_ingestion


In [20]:
type(data_ingestion_config)

__main__.DataIngestionConfig

In [ ]:

In trees.py:

